# Conditional Expectation

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
ndim = 2
gl.defineDefaultSpace(gl.ESpaceType.RN, ndim);

In [ ]:
fileNF = os.path.join(os.getenv('GSTLEARN_DATA'), "Scotland", "Scotland_Temperatures.NF")
dat = gl.Db.createFromNF(fileNF)
fileNF = os.path.join(os.getenv('GSTLEARN_DATA'), "Scotland", "Scotland_Elevations.NF")
grid = gl.DbGrid.createFromNF(fileNF)

In [ ]:
unique_neigh = gl.NeighUnique.create()

Histogram of the raw variable (Temperature)

In [ ]:
ax = gp.hist(dat,"January*", nbins=20, title="Temperatures")

Gaussian scores

In [ ]:
anam = gl.AnamHermite(30)
err = anam.fitFromLocator(dat)
err = anam.rawToGaussian(dat, "January_temp")

In [ ]:
anam.display()

Plot the Gaussian scores

In [ ]:
ax = gp.sortedcurve(dat["Y.January_temp"], dat["January_temp"],xlabel="Gaussian",ylabel="Raw")

Draw the histogram of the Gaussian transformed values

In [ ]:
ax = gp.hist(dat,"Y.January*", nbins=20, title="Temperatures (Gaussian scale)")

We calculate the experimental directional variogram of the gaussian scores and fit the Model (with the constraints that sill should be 1)

In [ ]:
varioparam = gl.VarioParam.createMultiple(ndir=2, npas=40, dpas=10)
vario_gauss2dir = gl.Vario.create(varioparam, dat)
err = vario_gauss2dir.compute()

fitmodgauss = gl.Model()
err = fitmodgauss.fit(vario_gauss2dir, 
                types=[gl.ECov.NUGGET, gl.ECov.SPHERICAL, gl.ECov.CUBIC],
                constraints = gl.Constraints(1))

In [ ]:
ax = gp.varmod(vario_gauss2dir, fitmodgauss)

In [ ]:
neighU = gl.NeighUnique.create()

Kriging of Gaussian scores

In [ ]:
err = gl.kriging(dat, grid, fitmodgauss, neighU)

In [ ]:
ax = gp.grid(grid, name="*estim", aspect=1, figsize=[8,8], title="Kriging of Gaussian scores")
ax = gp.point(dat, ax=ax)

In [ ]:
ax = gp.grid(grid, name="*stdev", aspect=1, figsize=[8,8], title="St. Dev. of Gaussian scores")
ax = gp.point(dat, ax=ax)

## Conditional Expectation

We use the Monte Carlo method with 1000 outcomes.

In [ ]:
selectivity = gl.Selectivity.createByKeys(["Z"], [], flag_est=True, flag_std=True)
err = gl.ConditionalExpectation(grid, anam, selectivity, "K*.estim", "K*.stdev", nbsimu=100,
                                namconv=gl.NamingConvention("CE",False,True,False))

Display of the Conditional Expectation

In [ ]:
ax = gp.grid(grid, name="CE*estim", aspect=1, figsize=[8,8],
            title = "Conditional Expectation")
ax = gp.point(dat, ax=ax)

Display of the Conditional Standard Deviation

In [ ]:
ax = gp.grid(grid, name="CE*stdev", aspect=1, figsize=[8,8],
            title="Conditional Standard Deviation")
ax = gp.point(dat, ax=ax)

Conditional Probability below 0

In [ ]:
selectivity = gl.Selectivity.createByKeys(["PROP"], zcuts=[0],flag_est=True, flag_std=True)
err = gl.ConditionalExpectation(grid, anam, selectivity, "K*.estim", "K*.stdev",
                                namconv=gl.NamingConvention("CE",False,True,False))

In [ ]:
ax = gp.grid(grid, name="CE.Proba*estim", aspect=1, figsize=[8,8],
            title = "Conditional Probability below 0")
ax = gp.point(dat, ax=ax)

Conditional Probability above 1

In [ ]:
selectivity = gl.Selectivity.createByKeys(["T"], zcuts=[1],flag_est=True, flag_std=True)
err = gl.ConditionalExpectation(grid, anam, selectivity, "K*.estim", "K*.stdev",
                                namconv=gl.NamingConvention("CE",False,True,False))

In [ ]:
ax = gp.grid(grid, name="CE.T*estim-1", aspect=1, figsize=[8,8],
            title = "Conditional Probability above 1")
ax = gp.point(dat, ax=ax)

In [ ]:
ax = gp.grid(grid, name="CE.T*stdev-1", aspect=1, figsize=[8,8],
            title = "Conditional probability (Standard Deviation)")
ax = gp.point(dat, ax=ax)